In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/242B_final_project') # customize this line to your working directory

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/242B_final_project/np_save/"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from skimage.transform import resize
import seaborn as sns
import torch
import torch.nn as nn
import torchvision.models as models
from torch.nn import functional as F
from torchvision.models import resnet50

import PIL.Image
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import random

In [ ]:
train_matched_pairs = np.load(f'{path}train_matched_pairs.npy', allow_pickle=True)
train_matched_labels = np.load(f'{path}train_matched_labels.npy', allow_pickle=True)
test_matched_pairs = np.load(f'{path}test_matched_pairs.npy', allow_pickle=True)
test_matched_labels = np.load(f'{path}test_matched_labels.npy', allow_pickle=True)

train_mismatched_pairs = np.load(f'{path}train_mismatched_pairs.npy', allow_pickle=True)
train_mismatched_labels = np.load(f'{path}train_mismatched_labels.npy', allow_pickle=True)
test_mismatched_pairs = np.load(f'{path}test_mismatched_pairs.npy', allow_pickle=True)
test_mismatched_labels = np.load(f'{path}test_mismatched_labels.npy', allow_pickle=True)

# Data Loader


In [ ]:
class SiameseDataset(Dataset):
    def __init__(self, matched_pairs, matched_labels, mismatched_pairs, mismatched_labels):
        self.pairs = np.concatenate((matched_pairs, mismatched_pairs), axis=0)
        self.labels = np.concatenate((matched_labels, mismatched_labels), axis=0)

        # Shuffle the data
        indices = np.arange(len(self.pairs))
        random.shuffle(indices)
        self.pairs = self.pairs[indices]
        self.labels = self.labels[indices]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img1 = self.pairs[idx][0]
        img2 = self.pairs[idx][1]
        label = self.labels[idx]

        return img1, img2, label

In [ ]:
train_dataset = SiameseDataset(train_matched_pairs, train_matched_labels,
                                train_mismatched_pairs, train_mismatched_labels)

test_dataset = SiameseDataset(test_matched_pairs, test_matched_labels,
                               test_mismatched_pairs, test_mismatched_labels)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# Model 1 - self defined (Siamese)

In [ ]:
def calculate_accuracy(data_loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in data_loader:
            img1, img2, labels = data
            img1, img2 = img1.to(device), img2.to(device)
            labels = labels.to(device)
            labels = labels.squeeze()
            # print(f'labels: {labels}')
            outputs = model(img1, img2).squeeze()
            # print(f'Probability: {outputs}')
            predicted = (outputs > 0.5).float()  # Ensures it's a float tensor
            # print(f'Predicted: {predicted}')
            correct += (predicted == labels.float()).sum().item()  # Cast labels to float for comparison
            # print(f'Correct: {correct}')
            total += labels.size(0)
            # print(f'Total: {total}')
    return 100 * correct / total

In [ ]:
import time
from torch.utils.data import DataLoader

def train_model(model, criterion, num_epochs, optimizer, train_loader, test_loader): #test_loader
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.train()  # Set the model to training mode

    # Logs for keeping track of loss and accuracy
    train_loss_log = []
    train_acc_log = []
    test_acc_log = []

    tic = time.time()

    for epoch in range(num_epochs):
        total_train_loss = 0
        model.train()  # Ensure model is in training mode

        for data in train_loader:
            img1, img2, labels = data
            img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward pass
            outputs = model(img1, img2)  # Adjusted to use the updated forward method
            loss = criterion(outputs, labels.float().view(-1, 1))

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

        # Calculate average loss and accuracy for the epoch
        avg_train_loss = total_train_loss / len(train_loader)
        train_accuracy = calculate_accuracy(train_loader, model, device)
        test_accuracy = calculate_accuracy(test_loader, model, device)

        train_loss_log.append(avg_train_loss)
        train_acc_log.append(train_accuracy)
        test_acc_log.append(test_accuracy)

        # Print statistics
        print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%')
        # print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%')

    toc = time.time()
    print("Elapsed Time : {:7.2f}".format(toc-tic))

    return train_loss_log, train_acc_log , test_acc_log


In [ ]:
# self define the model
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)  # Input is 224x224x3, output is 224x224x16
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1) # Input is 112x112x16, output is 112x112x32
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1) # Input is 56x56x32, output is 56x56x64

        # Pooling layer
        self.pool = nn.MaxPool2d(2, 2)

        # Fully connected layers
        self.fc1 = nn.Linear(64 * 28 * 28, 500)
        self.fc2 = nn.Linear(500, 256)
        self.fc3 = nn.Linear(256, 128)

        # Classification layer
        self.classifier = nn.Linear(128, 1)  # Outputs probability of match
        self.sigmoid = nn.Sigmoid()

    def forward_once(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)

        # Calculate the absolute difference between the outputs
        difference = torch.abs(output1 - output2)
        logits = self.classifier(difference)
        probability = self.sigmoid(logits)
        return probability

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseNetwork()
num_epochs = 10
criterion = nn.BCELoss()  # Ensure the label and output size match
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


train_loss_log, train_acc_log, test_acc_log= train_model(model, criterion, num_epochs, optimizer, train_loader, test_loader)

Epoch 1/10, Training Loss: 0.6939, Training Accuracy: 53.36%, Test Accuracy: 49.90%
Epoch 2/10, Training Loss: 0.6695, Training Accuracy: 56.73%, Test Accuracy: 54.40%
Epoch 3/10, Training Loss: 0.6488, Training Accuracy: 61.86%, Test Accuracy: 57.20%
Epoch 4/10, Training Loss: 0.6117, Training Accuracy: 71.82%, Test Accuracy: 62.30%
Epoch 5/10, Training Loss: 0.5647, Training Accuracy: 78.41%, Test Accuracy: 61.10%
Epoch 6/10, Training Loss: 0.5063, Training Accuracy: 82.32%, Test Accuracy: 61.20%
Epoch 7/10, Training Loss: 0.4071, Training Accuracy: 87.18%, Test Accuracy: 60.20%
Epoch 8/10, Training Loss: 0.3031, Training Accuracy: 93.14%, Test Accuracy: 60.40%
Epoch 9/10, Training Loss: 0.2037, Training Accuracy: 95.77%, Test Accuracy: 61.00%
Epoch 10/10, Training Loss: 0.1375, Training Accuracy: 96.64%, Test Accuracy: 62.30%
Elapsed Time : 3035.92


# Model 2 - ResNet18

In [ ]:
class ResNetModel(nn.Module):
    def __init__(self):
        super(ResNetModel, self).__init__()
        # Load a pre-trained ResNet and remove the last fully connected layer
        base_model = models.resnet18(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(base_model.children())[:-1])

        # Classification layer
        self.classifier = nn.Sequential(
            nn.Linear(512, 1),  # Adjust the input features to match the output of your feature extractor
            nn.Sigmoid()
        )

    def forward_once(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)  # Flatten the features
        return x

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)

        # Calculate the absolute difference between the outputs
        difference = torch.abs(output1 - output2)
        probability = self.classifier(difference)

        return probability

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNetModel()
num_epochs = 10
criterion = nn.BCELoss()  # Ensure the label and output size match
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


train_loss_log, train_acc_log, test_acc_log= train_model(model, criterion, num_epochs, optimizer, train_loader, test_loader)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 102MB/s]


Epoch 1/10, Training Loss: 0.6762, Training Accuracy: 65.59%, Test Accuracy: 60.40%
Epoch 2/10, Training Loss: 0.6195, Training Accuracy: 56.05%, Test Accuracy: 54.80%
Epoch 3/10, Training Loss: 0.5981, Training Accuracy: 71.41%, Test Accuracy: 62.90%
Epoch 4/10, Training Loss: 0.5468, Training Accuracy: 74.64%, Test Accuracy: 65.40%
Epoch 5/10, Training Loss: 0.4999, Training Accuracy: 74.05%, Test Accuracy: 66.00%
Epoch 6/10, Training Loss: 0.4775, Training Accuracy: 83.36%, Test Accuracy: 67.10%
Epoch 7/10, Training Loss: 0.4196, Training Accuracy: 83.14%, Test Accuracy: 66.70%
Epoch 8/10, Training Loss: 0.3420, Training Accuracy: 87.95%, Test Accuracy: 70.10%
Epoch 9/10, Training Loss: 0.2912, Training Accuracy: 90.68%, Test Accuracy: 65.00%
Epoch 10/10, Training Loss: 0.2340, Training Accuracy: 92.55%, Test Accuracy: 69.80%
Elapsed Time : 15517.45
